In [214]:
import json

def read_game_data(file_path):
    game_data = []
    with open(file_path, 'r') as file:
        for line in file:
            try:
                game_state = json.loads(line.strip())
                game_data.append(game_state)
            except json.JSONDecodeError:
                continue  # Skip lines that can't be decoded as JSON
    return game_data


In [215]:
def extract_features(game_data):
    extracted_data = []
    for state in game_data:
        new_state = {
            "turn_number": state["turn_number"],
            "current_player": state["current_player"],
            "num_players": state["num_players"],
            "cards_in_hands": state["cards_in_hands"],
            "playable_cards": state["playable_cards"],
            "known_opponent_cards": state["known_opponent_cards"],
            "num_cards_in_hands": state["num_cards_in_hands"],
            "top_card": state["top_card"],
            "cards_in_pile": state["cards_in_pile"],
            "face_up_cards": state["face_up_cards"],
            "num_face_up_cards": state["num_face_up_cards"],
            "num_face_down_cards": state["num_face_down_cards"],
            "play_direction": state["play_direction"],
            "num_cards_in_draw_pile": state["num_cards_in_draw_pile"]
        }
        extracted_data.append(new_state)
    return extracted_data


In [216]:
# Example usage
file_path = 'game_data.txt'
game_data = read_game_data(file_path)
extracted_data = extract_features(game_data)
print(extracted_data)


[{'turn_number': 0, 'current_player': 3, 'num_players': 6, 'cards_in_hands': [['Unknown', 'Unknown', 'Unknown'], ['Unknown', 'Unknown', 'Unknown'], ['6 Diamonds', '3 Hearts', '7 Diamonds'], ['Unknown', 'Unknown', 'Unknown'], ['Unknown', 'Unknown', 'Unknown'], ['Unknown', 'Unknown', 'Unknown']], 'playable_cards': '[[6 Diamonds, 3 Hearts, 7 Diamonds]]', 'known_opponent_cards': [[], [], [], [], [], []], 'num_cards_in_hands': [3, 3, 3, 3, 3, 3], 'top_card': None, 'cards_in_pile': [], 'face_up_cards': [['7 Hearts', '13 Hearts', '2 Clubs'], ['8 Hearts', '13 Diamonds', '14 Clubs'], ['12 Spades', '12 Hearts', '10 Spades'], ['12 Clubs', '12 Spades', '2 Hearts'], ['8 Diamonds', '8 Hearts', '10 Hearts'], ['11 Diamonds', '13 Spades', '2 Hearts']], 'num_face_up_cards': [3, 3, 3, 3, 3, 3], 'num_face_down_cards': [3, 3, 3, 3, 3, 3], 'play_direction': 1, 'num_cards_in_draw_pile': 54}, {'turn_number': 1, 'current_player': 4, 'num_players': 6, 'cards_in_hands': [['Unknown', 'Unknown', 'Unknown'], ['Unkn

In [217]:
def get_line_from_extracted_data(extracted_data, line_number):
    if line_number < 0 or line_number >= len(extracted_data):
        return "Line number out of range"
    return extracted_data[line_number]

# Example usage
line_number = 102  # Get the first line of extracted data
line_data = get_line_from_extracted_data(extracted_data, line_number)
print(line_data)

{'turn_number': 102, 'current_player': 1, 'num_players': 6, 'cards_in_hands': [['14 Spades', '6 Spades', '7 Spades'], ['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'], [], ['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'], [], ['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown']], 'playable_cards': '[13 Diamonds, 14 Diamonds, Joker, 14 Clubs]', 'known_opponent_cards': [['6 Spades', '7 Spades'], ['14 Diamonds', 'Joker', '14 Clubs'], [], ['5 Spades', '6 Diamonds', '6 Clubs', '6 Hearts', '9 Clubs', '5 Clubs', '7 Diamonds', '7 Diamonds', '6 Diamonds', '7 Clubs', '6 Spades', 

In [218]:
def get_selected_features(extracted_data, line_number):
    if line_number < 1 or line_number >= len(extracted_data):  # Check if line number is valid
        return "Line number out of range"

    current_player = extracted_data[line_number - 1]["current_player"] - 1  # Get the current player index from the previous line
    selected_features = {
        "top_card": extracted_data[line_number]["top_card"],
        "num_cards_in_hand": extracted_data[line_number]["num_cards_in_hands"][current_player],
        "num_face_up_cards": extracted_data[line_number]["num_face_up_cards"][current_player]
    }
    return selected_features

# Example usage
line_number = 102  # Get the second line of extracted data
selected_data = get_selected_features(extracted_data, line_number)
print(selected_data)


{'top_card': '13 Diamonds', 'num_cards_in_hand': 5, 'num_face_up_cards': 3}


In [219]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from joblib import dump

# Function to extract rank and suit from a card
def extract_rank_suit(card):
    if card is None or ' ' not in card:
        return [0, 'None']  # Handle cases where top_card is None or doesn't contain a space
    rank, suit = card.split()  # Split the card into rank and suit
    return [int(rank), suit]  # Convert rank to integer


# Prepare the dataset with rank and suit
X = []
y = []
for i in range(1, len(extracted_data) - 1):
    current_features = get_selected_features(extracted_data, i)
    next_features = get_selected_features(extracted_data, i + 1)
    rank, suit = extract_rank_suit(current_features['top_card'])
    X.append([rank, suit, current_features['num_cards_in_hand'], current_features['num_face_up_cards']])
    y.append([next_features['num_cards_in_hand'], next_features['num_face_up_cards']])

# Define a transformer for the suit feature
transformer = ColumnTransformer(
    transformers=[
        ('suit_encoder', OneHotEncoder(), [1])  # Apply OneHotEncoder to the suit column
    ],
    remainder='passthrough'  # Keep the other columns unchanged
)

# Apply the transformation
X_transformed = transformer.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Save the model to a file
dump(model, 'random_forest_model.joblib')


Mean Squared Error: 14.179196567851946


['random_forest_model.joblib']

In [220]:
import numpy as np

# Assume we have a function to encode the state and action into indices for the Q-table
def encode_state(state):
    # Implement a way to encode the state into an index
    pass

def encode_action(action, actions):
    # Implement a way to encode the action into an index
    return actions.index(action)

# Initialize the Q-table
num_states = ...  # Define the number of unique states
num_actions = ...  # Define the number of unique actions
q_table = np.zeros((num_states, num_actions))

# Define the learning parameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate

# Q-learning algorithm
for episode in range(num_episodes):
    state = ...  # Initialize the state based on the current game state
    state_index = encode_state(state)
    done = False

    while not done:
        if np.random.uniform(0, 1) < epsilon:
            action_index = np.random.choice(range(num_actions))  # Explore action space
        else:
            action_index = np.argmax(q_table[state_index])  # Exploit learned values

        action = ...  # Map the action_index back to a playable card
        new_state, reward, done = ...  # Take the action and observe the new state and reward
        new_state_index = encode_state(new_state)

        q_table[state_index, action_index] = q_table[state_index, action_index] + alpha * (reward + gamma * np.max(q_table[new_state_index]) - q_table[state_index, action_index])
        state_index = new_state_index


NameError: name 'max_num_cards' is not defined